In [160]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [161]:
!ls "/content/gdrive/My Drive/2024 Hackathon"

'Copy of Brainstorming.gdoc'	'Copy of prompt.pdf'		    fangraphs_season_level.csv
'Copy of data_dictionary.xlsx'	'Copy of RedsHackathonCode.ipynb'   savant_pitch_level.csv


In [162]:
# Imports and Setupd
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [163]:
# fan_df contains the season level data for each pitcher since 2021
fan_df = pd.read_csv('/content/gdrive/My Drive/2024 Hackathon/fangraphs_season_level.csv')

In [164]:
# savant_df contains the pitch level data for each pitch since 2021
savant_df = pd.read_csv('/content/gdrive/My Drive/2024 Hackathon/savant_pitch_level.csv')

In [165]:
fan_df.head()

,PlayerId,MLBAMID,Name,NameASCII,Throws,Season,Age,Team,Role,G,...,Pit_plus_FC,Stf_plus_FS,Loc_plus_FS,Pit_plus_FS,Stf_plus_FO,Loc_plus_FO,Pit_plus_FO,Stuff_plus,Location_plus,Pitching_plus
0,18,491703,NeftalÃ­ Feliz,Neftali Feliz,R,2021,33,- - -,RP,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.316418,95.123881,95.873134
1,1157,488984,Tommy Hunter,Tommy Hunter,R,2021,34,NYM,RP,3,...,103.100000,NaN,NaN,NaN,NaN,NaN,NaN,86.285556,94.884444,95.112222
2,1159,461865,Andrew Romine,Andrew Romine,R,2021,35,CHC,RP,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.925000,108.425000,99.200000
3,1246,456713,Matt Bush,Matt Bush,R,2021,35,TEX,RP,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.852174,98.340580,102.611594
4,1247,493603,Adam Ottavino,Adam Ottavino,R,2021,35,BOS,RP,69,...,66.583333,NaN,NaN,NaN,NaN,NaN,NaN,116.269527,94.625394,102.238617


In [166]:
savant_df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,role_key,sp_indicator,rp_indicator,pitch_number_appearance,pitcher_at_bat_number,times_faced
0,FF,2021-04-05,93.2,0.84,5.74,"Duffy, Danny",592696,518633,strikeout,swinging_strike,...,Standard,166.0,-0.014,-0.134,SP,1,0,97,22,3
1,SL,2021-04-05,82.9,1.03,5.69,"Duffy, Danny",592696,518633,NaN,foul,...,Standard,309.0,0.000,0.000,SP,1,0,96,22,3
2,FF,2021-04-05,94.6,-1.85,5.77,"Foltynewicz, Mike",543760,592314,caught_stealing_3b,ball,...,Standard,220.0,0.014,-0.202,SP,1,0,95,19,3
3,FF,2021-04-05,97.1,1.86,6.57,"Rodón, Carlos",657108,607074,field_out,hit_into_play,...,Standard,147.0,-0.007,-0.264,SP,1,0,95,22,3
4,FF,2021-04-05,93.6,0.60,5.85,"Duffy, Danny",592696,518633,NaN,ball,...,Standard,164.0,0.000,0.043,SP,1,0,95,22,3


In [167]:
def ks(events):
  ks = 0
  for event in events:
    if event == "strikeout":
      ks += 1
  return ks

def bbs(events):
  bbs = 0
  for event in events:
    if event in ["walk", "hit_by_pitch"] :
      bbs += 1
  return bbs
def gbs(bb_type):
  gbs = 0
  for hit in bb_type:
    if hit == "ground_ball":
      gbs += 1
  return gbs

def fbs(bb_type):
  fbs = 0
  for hit in bb_type:
    if hit == "fly_ball":
      fbs += 1
  return fbs

def pus(bb_type):
  pus = 0
  for hit in bb_type:
    if hit == "popup":
      pus += 1
  return pus

def pas(pitch_number):
  pas = 0
  for pitch in pitch_number:
    if pitch == 1:
      pas += 1
  return pas

In [168]:
savant_df_2023 = savant_df[savant_df['game_year'] == 2023]
savant_df_sorted = savant_df_2023.sort_values(["pitcher", "times_faced"])
savant_appearance_groups = savant_df_sorted.groupby(["pitcher", "times_faced"]).agg(
  PA= ("pitch_number", pas),
  SO= ("events", ks),
  BB= ("events", bbs),
  GB= ("bb_type", gbs),
  FB= ("bb_type", fbs),
  PU= ("bb_type", pus)
)

savant_appearance_groups["PA"] = savant_appearance_groups["PA"].astype(float)
savant_appearance_groups["SO"] = savant_appearance_groups["SO"].astype(float)
savant_appearance_groups["BB"] = savant_appearance_groups["BB"].astype(float)
savant_appearance_groups["GB"] = savant_appearance_groups["GB"].astype(float)
savant_appearance_groups["FB"] = savant_appearance_groups["FB"].astype(float)
savant_appearance_groups["PU"] = savant_appearance_groups["PU"].astype(float)

In [169]:
def SIERA_calc(row):
  neg = 1
  if (row["GB"] - row["FB"] - row["PU"]) > 0:
    neg = -1
  return (6.145 - 16.986*(row["SO"] / row["PA"]) + 11.434*(row["BB"] / row["PA"]) - 1.858*((row["GB"] - row["FB"] - row["PU"])/row["PA"]) + 7.653*((row["SO"] / row["PA"])**2) + neg * 6.664*(((row["GB"] - row["FB"] - row["PU"]) / row["PA"])**2) + 10.130*(row["SO"] / row["PA"])*((row["GB"] - row["FB"] - row["PU"])/row["PA"]) - 5.195*(row["BB"]/row["PA"])*((row["GB"] - row["FB"] - row["PU"]) / row["PA"]))

In [170]:
savant_appearance_groups["SIERA"] = savant_appearance_groups.apply(SIERA_calc, axis= 1)
print(savant_appearance_groups.head(20))
graph_df = savant_appearance_groups[savant_appearance_groups["PA"] >= 10]
graph_df = graph_df.groupby([graph_df.index.get_level_values(0), graph_df.index.get_level_values(1)]).agg({
    "SIERA": "mean"
})
print(graph_df.head(20))
graph_df.reset_index()

# new_df = graph_df.pivot(index = 'pitcher', columns = {'times_faced'}, values = 'SIERA')
# print(new_df.head(20))

                        PA    SO    BB    GB    FB    PU     SIERA
pitcher times_faced                                               
425794  1            187.0  26.0  17.0  57.0  36.0   7.0  4.864343
        2            178.0  14.0  12.0  67.0  42.0   6.0  5.400604
        3            113.0  15.0  12.0  34.0  31.0   1.0  5.218366
        4              2.0   0.0   0.0   1.0   1.0   0.0  6.145000
425844  1            271.0  50.0  18.0  89.0  53.0  13.0  3.954668
        2            251.0  39.0   8.0  89.0  50.0   8.0  3.897755
        3             72.0   6.0   1.0  26.0  18.0   4.0  4.860551
        4              1.0   0.0   0.0   1.0   0.0   0.0 -2.377000
434378  1            244.0  48.0  22.0  64.0  52.0  19.0  4.145647
        2            242.0  57.0  17.0  64.0  41.0  20.0  3.372949
        3            174.0  38.0  11.0  42.0  40.0  18.0  3.577227
        4              8.0   1.0   0.0   2.0   1.0   1.0  4.141328
445276  1            186.0  52.0  17.0  25.0  44.0  21.0  3.24

<ipython-input-169-fe736e34f956>:5: RuntimeWarning: invalid value encountered in double_scalars
  return (6.145 - 16.986*(row["SO"] / row["PA"]) + 11.434*(row["BB"] / row["PA"]) - 1.858*((row["GB"] - row["FB"] - row["PU"])/row["PA"]) + 7.653*((row["SO"] / row["PA"])**2) + neg * 6.664*(((row["GB"] - row["FB"] - row["PU"]) / row["PA"])**2) + 10.130*(row["SO"] / row["PA"])*((row["GB"] - row["FB"] - row["PU"])/row["PA"]) - 5.195*(row["BB"]/row["PA"])*((row["GB"] - row["FB"] - row["PU"]) / row["PA"]))
<ipython-input-169-fe736e34f956>:5: RuntimeWarning: divide by zero encountered in double_scalars
  return (6.145 - 16.986*(row["SO"] / row["PA"]) + 11.434*(row["BB"] / row["PA"]) - 1.858*((row["GB"] - row["FB"] - row["PU"])/row["PA"]) + 7.653*((row["SO"] / row["PA"])**2) + neg * 6.664*(((row["GB"] - row["FB"] - row["PU"]) / row["PA"])**2) + 10.130*(row["SO"] / row["PA"])*((row["GB"] - row["FB"] - row["PU"])/row["PA"]) - 5.195*(row["BB"]/row["PA"])*((row["GB"] - row["FB"] - row["PU"]) / row["PA

,pitcher,times_faced,SIERA
0,425794,1,4.864343
1,425794,2,5.400604
2,425794,3,5.218366
3,425844,1,3.954668
4,425844,2,3.897755
...,...,...,...
1387,696147,1,4.267591
1388,700363,1,5.750572
1389,700363,2,3.583681
1390,700363,3,2.129657


In [171]:
SIERA_df = pd.DataFrame({'id': graph_df.index.get_level_values(0), 'times_faced': graph_df.index.get_level_values(1)})
SIERA_df['SIERA'] = graph_df['SIERA'].values
SIERA_corr_df = SIERA_df.groupby('id')[['times_faced', 'SIERA']].corr().unstack().iloc[:, 1]
SIERA_corr_df = SIERA_corr_df.reset_index()
SIERA_corr_df.columns = ['pitcher', 'SIERA_corr']
SIERA_corr_df.head(15)
fan_df = pd.merge(fan_df, SIERA_corr_df, how = 'inner', left_on = 'MLBAMID', right_on = 'pitcher')

In [172]:
# fan_df = fan_df[fan_df['Season'] == 2023]
fan_df['SIERA_corr'] = fan_df['SIERA_corr'].fillna(0)
fan_df = fan_df.reset_index()

In [173]:
# Function for entropy given row of pitches
def disToEnt(row):
  probArr = row / sum(row)
  entropy = (-1.0 * np.sum(probArr * np.log2(probArr + 1e-10)))
  return entropy

In [174]:
# Add feature for pitch_type entropy
# Filter for left-handed batters
l_savant_df = savant_df[savant_df['stand'] == 'L'].groupby('pitcher')['pitch_type'].value_counts().unstack(fill_value=0)
# Calculate pitch entropy for left-handed pitchers
l_savant_df['l_pitch_entropy'] = l_savant_df.apply(disToEnt, axis=1)

# Filter for right-handed batters
r_savant_df = savant_df[savant_df['stand'] == 'R'].groupby('pitcher')['pitch_type'].value_counts().unstack(fill_value=0)
# Calculate pitch entropy for right-handed pitchers
r_savant_df['r_pitch_entropy'] = r_savant_df.apply(disToEnt, axis=1)
# Merge left and right-handed pitch entropies on 'pitcher'
by_pitcher_savant_df = pd.merge(l_savant_df, r_savant_df['r_pitch_entropy'], on='pitcher', how='inner')
# Left join fan and savant on 'PlayerId'
# Check for duplicates in merging columns
fan_df = pd.merge(fan_df, by_pitcher_savant_df[['l_pitch_entropy', 'r_pitch_entropy']], left_on='MLBAMID', right_on='pitcher', how='inner')

del l_savant_df, r_savant_df, by_pitcher_savant_df
# two columns (left entropy and right entropy)
# model platoon split
# integrate mean Stuff+ for each pitch into entropy

In [175]:
# Remove examples where (IP + G)
fan_df = fan_df[(fan_df['G'] > 5) | (fan_df['IP'] > 20)]

In [177]:
# Correlation between velo and pitch #
# Group by pitcher then get corr between pitch # and velo
correlation_per_pitcher = savant_df.groupby('pitcher')[['pitch_number_appearance', 'release_speed']].corr().unstack().iloc[:, 1]

# Resetting the index to have 'pitcher' as a column
correlation_per_pitcher = correlation_per_pitcher.reset_index()
correlation_per_pitcher.columns = ['pitcher', 'velo_pitch_corr']

# Merging the correlation results into fan_df based on pitcher's unique ID
fan_df = pd.merge(fan_df, correlation_per_pitcher, how='inner', left_on='MLBAMID', right_on='pitcher')

In [178]:
# Stuff+ per pitch type weighted by pitch type frequency
savant_df_2023 = savant_df_2023.groupby('pitcher')['pitch_type'].value_counts().unstack(fill_value = 0)
fan_df = pd.merge(fan_df, savant_df_2023, how = 'inner', left_on = 'MLBAMID', right_on = 'pitcher')

In [184]:
pitch_lookup = {
  # index in pitch_type array to threshold
  0 : (279, np.mean(fan_df.iloc[:, 279]) + .5 * np.std(fan_df.iloc[:, 279])),
  1 : (282, np.mean(fan_df.iloc[:, 282]) + .5 * np.std(fan_df.iloc[:, 282])),
  2 : (282, np.mean(fan_df.iloc[:, 282]) + .5 * np.std(fan_df.iloc[:, 282])),
  3 : (279, np.mean(fan_df.iloc[:, 279]) + .5 * np.std(fan_df.iloc[:, 279])),
  4 : (285, np.mean(fan_df.iloc[:, 285]) + .5 * np.std(fan_df.iloc[:, 285])),
  5 : (297, np.mean(fan_df.iloc[:, 297]) + .5 * np.std(fan_df.iloc[:, 297])),
  6 : (285, np.mean(fan_df.iloc[:, 285]) + .5 * np.std(fan_df.iloc[:, 285])),
  7 : (279, np.mean(fan_df.iloc[:, 279]) + .5 * np.std(fan_df.iloc[:, 279])),
  8 : (279, np.mean(fan_df.iloc[:, 279]) + .5 * np.std(fan_df.iloc[:, 279])),
  9 : (294, np.mean(fan_df.iloc[:, 294]) + .5 * np.std(fan_df.iloc[:, 294])),
  10 : (279, np.mean(fan_df.iloc[:, 279]) + .5 * np.std(fan_df.iloc[:, 279])),
  11 : (285, np.mean(fan_df.iloc[:, 285]) + .5 * np.std(fan_df.iloc[:, 285])),
  12 : (279, np.mean(fan_df.iloc[:, 279]) + .5 * np.std(fan_df.iloc[:, 279])),
  13 : (288, np.mean(fan_df.iloc[:, 288]) + .5 * np.std(fan_df.iloc[:, 288])),
  14 : (291, np.mean(fan_df.iloc[:, 291]) + .5 * np.std(fan_df.iloc[:, 291])),
  15 : (291, np.mean(fan_df.iloc[:, 291]) + .5 * np.std(fan_df.iloc[:, 291])),
  16 : (282, np.mean(fan_df.iloc[:, 282]) + .5 * np.std(fan_df.iloc[:, 282]))
}

def num_elite(row):
  num_elite = 0
  for i, p_num in enumerate(row[315:]):
    if p_num > 10 and row[pitch_lookup[i][0]] > pitch_lookup[i][1]:
      num_elite += 1
  return num_elite

In [185]:
fan_df['num_elite'] = fan_df.apply(num_elite, axis = 1)

In [195]:
fan_df_r = fan_df[fan_df['Role'] == 'RP']
fan_df_s = fan_df[fan_df['Role'] == 'SP']

In [196]:
# Creating starter and reliever dataframe
fin_df_r = fan_df_r[fan_df_r['Season'] == 2023][['MLBAMID', 'NameASCII', 'l_pitch_entropy', 'r_pitch_entropy', 'velo_pitch_corr', 'SIERA_corr', 'SIERA', 'num_elite', 'K_minus_BB_pct', 'K_pct']]
fin_df_s = fan_df_s[fan_df_s['Season'] == 2023][['MLBAMID', 'NameASCII', 'l_pitch_entropy', 'r_pitch_entropy', 'velo_pitch_corr', 'SIERA_corr', 'SIERA', 'num_elite', 'K_minus_BB_pct', 'K_pct']]

In [197]:
# Standardize rows
for column in ['l_pitch_entropy', 'r_pitch_entropy', 'velo_pitch_corr', 'SIERA_corr', 'SIERA', 'num_elite', 'K_minus_BB_pct']:
  fin_df_s[column] = (fin_df_s[column] - fan_df_s[column].mean()) / fan_df_s[column].std()
  if column != 'SIERA_corr':
    fin_df_r[column] = (fin_df_r[column] - fan_df_r[column].mean()) / fan_df_r[column].std()

In [198]:
def s_score_starter(row):

  s_score = (.1 * row[2] + .15 * row[3] - .2 *(abs(row[3] - row[2])) + .4 * row[4] + -.4 * row[5] + -.25 * row[6] + .2 * row[7] + .1 * row[8]) / 8
  return s_score

In [199]:
def r_score_reliever(row):

  r_score = (.1 * row[2] + .1 * row[3] -.25 * row[6] + .2 * row[7] + .1 * row[8] + .25 * row[9]) / 6
  return r_score

In [200]:
fin_df_s['s_pot'] = fin_df_s.apply(s_score_starter, axis = 1)
fin_df_s['r_pot'] = fin_df_s.apply(r_score_reliever, axis = 1)
fin_df_r['s_pot'] = fin_df_r.apply(s_score_starter, axis = 1)
fin_df_r['r_pot'] = fin_df_r.apply(r_score_reliever, axis = 1)

In [204]:
fin_df_r['dif_pot'] = fin_df_r['s_pot'] - fin_df_r['r_pot']
fin_df_s['dif_pot'] = fin_df_s['s_pot'] - fin_df_s['r_pot']

In [217]:
fin_df_r.sort_values(by = 'dif_pot', ascending = False).head(20)

,MLBAMID,NameASCII,l_pitch_entropy,r_pitch_entropy,velo_pitch_corr,SIERA_corr,SIERA,num_elite,K_minus_BB_pct,K_pct,s_pot,r_pot,dif_pot
972,670097,Zack Short,-1.255324,-1.797477,6.355032,0.000000,2.754986,-1.042435,-1.910894,0.0313,0.118763,-0.230963,0.349726
478,642545,Jaime Barria,0.535901,0.110512,3.073734,-0.988895,1.103541,-1.042435,-0.970752,0.1626,0.128587,-0.079359,0.207946
1500,687922,Easton Lucas,-0.498106,0.352628,2.806735,0.000000,0.859459,-1.042435,-0.744900,0.2000,0.057224,-0.077065,0.134289
98,571710,Mychal Givens,-0.698168,-0.462570,1.289665,0.000000,5.219766,-0.072354,-4.384380,0.0909,-0.178538,-0.308533,0.129995
1409,657514,Brennan Bernardino,-1.514726,-1.228319,2.445969,0.000000,-0.268113,-1.042435,0.229256,0.2472,0.058357,-0.055173,0.113530
1497,686747,Joe La Sorsa,-1.419796,-0.697855,1.302065,-1.000000,0.213496,-1.042435,-0.128570,0.1931,0.031883,-0.073035,0.104917
1492,678606,Jose A. Ferrer,-2.019353,-1.710375,1.986712,0.000000,0.227345,-1.042435,-0.761227,0.1761,-0.008381,-0.111732,0.103351
1037,686654,Ty Adcock,-1.488338,-1.603003,2.323776,0.000000,-0.329044,-0.072354,0.670075,0.1897,0.081511,-0.021152,0.102663
1213,657612,Tim Hill,-0.193523,0.013281,1.762839,0.000000,0.391646,-1.042435,-1.102726,0.1287,0.028718,-0.067087,0.095805
664,664776,Jake Cousins,-0.170149,-0.846753,1.324308,0.000000,2.875107,-1.042435,-2.818383,0.1556,-0.119841,-0.211982,0.092141


In [218]:
fin_df_s.sort_values(by = 'dif_pot').head(25)

,MLBAMID,NameASCII,l_pitch_entropy,r_pitch_entropy,velo_pitch_corr,SIERA_corr,SIERA,num_elite,K_minus_BB_pct,K_pct,s_pot,r_pot,dif_pot
594,669330,Tyler Wells,-0.032251,0.575425,-2.507309,1.008601,-0.057454,0.143192,0.554322,0.2489,-0.168297,0.035829,-0.204126
1404,640455,Sean Manaea,0.943121,0.570667,-2.297554,1.015579,0.012448,-0.788292,0.123949,0.2255,-0.171026,0.009896,-0.180922
1536,554430,Zack Wheeler,-0.395809,0.647662,-1.526216,0.733256,-1.067583,2.006160,1.309105,0.2694,-0.031985,0.148596,-0.180580
711,656876,Drew Rasmussen,0.500867,0.906573,-2.138851,0.534775,-1.159991,1.074676,1.041752,0.2655,-0.044426,0.136038,-0.180464
1539,594798,Jacob deGrom,-0.898816,-1.960039,-1.704895,-0.203749,-2.807237,2.006160,3.537588,0.3913,0.032526,0.211457,-0.178930
1640,680694,Kyle Bradish,0.640123,0.535887,-1.454883,0.807850,-0.765326,2.006160,0.738535,0.2504,-0.014391,0.141103,-0.155494
1059,686973,Louie Varland,-0.275824,-0.166428,-1.942747,0.988539,-0.142767,-0.788292,0.296751,0.2241,-0.167404,-0.013415,-0.153989
1333,666205,Kyle Muller,-1.258423,-0.373405,-2.119611,0.997342,1.483382,-0.788292,-1.439412,0.1570,-0.284760,-0.132730,-0.152031
1005,671737,Taj Bradley,-0.093900,-0.471188,-1.388030,0.962122,-0.699690,0.143192,0.950461,0.2844,-0.099622,0.052200,-0.151822
1041,682990,Quinn Priester,1.088483,-0.599080,-1.615232,0.818115,1.609142,-0.788292,-1.763822,0.1563,-0.253524,-0.108052,-0.145472
